<a href="https://colab.research.google.com/github/alexcpass/EBAC_alexcpass/blob/main/Profissao_Analista_de_dados_M28_Exercicio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br> 
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

# **COVID Dashboard**

## 1\. Contexto

A análise de dados para um arquivo sobre a COVID-19 envolve várias etapas importantes. Primeiro, é necessário coletar e organizar os dados em um formato adequado para análise. Esse trabalho tem por objetivo analisar dados em um periodo especifico (2020-2021).Os dados sobre vacinação da COVID-19 serão obtidos da Universidade John Hopkins. O objetivo é realizar um comparativo entre Brasil e Austrália no mesmo período (Janeiro/2021)

## 2\. Pacotes e bibliotecas

In [174]:
import math
from typing import Iterator
from datetime import datetime, timedelta
import requests
import numpy as np
import pandas as pd

## 3\. Extração

In [175]:
dados = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [176]:
dados.to_csv('dados.csv', sep=',', index=False)

In [177]:
dados.head(5)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


In [178]:
dados.shape

(4012, 14)

## 4\. Transformação

Vou transformar a coluna Last Updade apenas em mês-dia-ano

Filtrar apenas os dados referentes ao Brasil e Australia

In [179]:
paises_filtrados = ['Brazil', 'Australia']
df_paises = dados.loc[dados['Country_Region'].isin(paises_filtrados)]

In [180]:
df_paises.shape

(35, 14)

In [181]:
df_pais = df_paises.drop(['FIPS', 'Admin2','Combined_Key'], axis=1)

In [182]:
df_pais.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio
8,Australian Capital Territory,Australia,2021-01-13 05:22:15,-35.4735,149.0124,118,3,115,0,27.563653,2.542373
9,New South Wales,Australia,2021-01-13 05:22:15,-33.8688,151.2093,5041,54,0,4987,62.096576,1.071216
10,Northern Territory,Australia,2021-01-13 05:22:15,-12.4634,130.8456,91,0,74,17,37.052117,0.000000
11,Queensland,Australia,2021-01-13 05:22:15,-27.4698,153.0251,1283,6,1245,32,25.080637,0.467654
12,South Australia,Australia,2021-01-13 05:22:15,-34.9285,138.6007,590,4,574,12,33.589525,0.677966


In [183]:
# Convertendo a coluna para datetime
df_pais['Last_Update'] = pd.to_datetime(df_pais['Last_Update'])

# Separando em colunas de ano, mes e dia
df_pais['ano'] = df_pais['Last_Update'].dt.year
df_pais['mes'] = df_pais['Last_Update'].dt.month
df_pais['dia'] = df_pais['Last_Update'].dt.day

In [184]:
df_pais.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,ano,mes,dia
8,Australian Capital Territory,Australia,2021-01-13 05:22:15,-35.4735,149.0124,118,3,115,0,27.563653,2.542373,2021,1,13
9,New South Wales,Australia,2021-01-13 05:22:15,-33.8688,151.2093,5041,54,0,4987,62.096576,1.071216,2021,1,13
10,Northern Territory,Australia,2021-01-13 05:22:15,-12.4634,130.8456,91,0,74,17,37.052117,0.000000,2021,1,13
11,Queensland,Australia,2021-01-13 05:22:15,-27.4698,153.0251,1283,6,1245,32,25.080637,0.467654,2021,1,13
12,South Australia,Australia,2021-01-13 05:22:15,-34.9285,138.6007,590,4,574,12,33.589525,0.677966,2021,1,13


In [186]:
df_pais.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,Case_Fatality_Ratio,ano,mes,dia
8,Australian Capital Territory,Australia,2021-01-13 05:22:15,-35.4735,149.0124,118,3,115,0,27.563653,2.542373,2021,1,13
9,New South Wales,Australia,2021-01-13 05:22:15,-33.8688,151.2093,5041,54,0,4987,62.096576,1.071216,2021,1,13
10,Northern Territory,Australia,2021-01-13 05:22:15,-12.4634,130.8456,91,0,74,17,37.052117,0.000000,2021,1,13
11,Queensland,Australia,2021-01-13 05:22:15,-27.4698,153.0251,1283,6,1245,32,25.080637,0.467654,2021,1,13
12,South Australia,Australia,2021-01-13 05:22:15,-34.9285,138.6007,590,4,574,12,33.589525,0.677966,2021,1,13


In [187]:
df_pais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 8 to 67
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Province_State       35 non-null     object        
 1   Country_Region       35 non-null     object        
 2   Last_Update          35 non-null     datetime64[ns]
 3   Lat                  35 non-null     float64       
 4   Long_                35 non-null     float64       
 5   Confirmed            35 non-null     int64         
 6   Deaths               35 non-null     int64         
 7   Recovered            35 non-null     int64         
 8   Active               35 non-null     int64         
 9   Incident_Rate        35 non-null     float64       
 10  Case_Fatality_Ratio  35 non-null     float64       
 11  ano                  35 non-null     int64         
 12  mes                  35 non-null     int64         
 13  dia                  35 non-null     

In [188]:
df_pais.to_csv('df_pais.csv', sep=',', index=False)

## 5\. Carregamento

Utilizando mais uma vez o Power BI para as visualizações


Segue o link do trabalho:

https://app.powerbi.com/links/UxorIb4QdA?ctid=717c3155-980a-45ce-b20c-ab6d0e642d11&pbi_source=linkShare